# Deep learning framework example

This notebook demonstrates how to use the deeplearning API to train and test the model on the [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris).

To install the necessary software, use `make deps` in the root directory of the project.  If you don't have make installed or don't want to use it, then use:

`pip install -r src/python/requirements.txt`

**Note**: it is not necessary to install the `zensols.deeplearn` package to run this notebook.


## What is demoed

This notebook shows how to create an executor directly.  However, an easier more client friendly facade is available and given in the *mnist* notebook.

Also see the the *debug* notebook, which demostrates how to debug a model.

In [ ]:
# set up notebook environment
import sys
app_root_dir = '..'
sys.path.append(app_root_dir + '/src/python')
sys.path.append(app_root_dir + '/test/python')

## Create the object factory

This creates a factoty that instantiates Python objects using a simple configuration (INI).  This removes much of the complexity of creating and "hooking up" all the instances.

In [ ]:
import logging
from pathlib import Path
from zensols.config import ExtendedInterpolationEnvConfig as AppConfig
from zensols.config import ImportConfigFactory
from zensols.deeplearn import TorchConfig
from zensols.deeplearn.result import ModelResultGrapher

# initialze PyTorch and set the random seed so things are predictable
TorchConfig.init()

temp_dir = Path('../target')
if temp_dir.is_dir():
    import shutil
    print(f'removing previous results in {temp_dir}')
    shutil.rmtree(temp_dir)

# configure logging
logging.basicConfig(format='%(asctime)-15s %(name)s [%(levelname)s]: %(message)s',
                    level=logging.WARNING)
for name in ['zensols.deeplearn.result',
             'zensols.deeplearn.model.facade',
             'zensols.deeplearn.batch.stash']:
    logging.getLogger(name).setLevel(logging.INFO)

# configure the environment
config = AppConfig(app_root_dir + '/test-resources/iris/iris.conf',
                   env={'app_root': app_root_dir})

# create a factoty that instantiates Python objects using a simple configuration (INI)
factory = ImportConfigFactory(config)

## Create the model executor

Use the factory to create the model executor (see the `executor` section of `test-resources/iris/iris.conf`).  The `write` method gives statistics on the data set that is configured on the executor.

In [ ]:
executor = factory('executor')
executor.write()

## Print the model

Every time the executor is told to train, it creates a new model.  It also stores this model to the disk every time the validation loss drops.  The method that controls the creation is `_create_model`, and not meant to be called by the client.  Note this creates a new PyTorch `torch.nn.Module` every time and isn't the same instance used by the executor.

In this case, a four deep fully connected network is created and fanned out from the 4 features (from four columns from the CSV file) to 20.  This is then reduce to the output layer having three neurons indicating which type of flower (setosa, versicolor, or virginica).

In [ ]:
print(executor._create_model())

## Train and test the model

This trains the model on the Iris (flower) data set and prints the results.  The PyTorch model itself is also printed.

In [ ]:
# tell the executor to give us console output
executor.progress_bar = True

# train the model
executor.train()

# test the model
res = executor.test()

# write a summary of the results
res.write()

# graph the results
grapher = ModelResultGrapher('Iris dataset', [15, 5])
grapher.plot([res])

## Create the model facade

An easier wasy to use the executor is with a *facade*.  A `ModelFacade` provides easy to use client entry points to the model executor, which trains, validates, tests, saves and loads the model.  Create the facade with a factory, which in turn creates the executor.  The statistics on the data set that is configured on the executor is, by default, printed to standard out.  You can set the `writer` property to `None` on the facade to disable this.

In [ ]:
from dataclasses import dataclass
from zensols.deeplearn.model import ModelFacade

@dataclass
class IrisModelFacade(ModelFacade):
    def _configure_debug_logging(self):
        super()._configure_debug_logging()
        logging.getLogger('iris.model').setLevel(logging.DEBUG)
        
# deallocate the previous executor
executor.deallocate()
# create the facade
facade = IrisModelFacade(config)

## Fine tune setting hyperparameters

Now that we have our model training and we have evaluated the results, we see the validation loss is very spiky.  This means our learning rate is probably too high as when it moves during stochastic gradient descent it is "jumping" too far and back up the error surface.

To fix that, let's decrease the learning rate.  We can do that by adjusting the hyperparameters directly on the facade.  In fact, the purpose of the facade is to make changes easily such as this.

Notice that the facade prints the output with a correctly configured scroll bar by default.  Output can be disabled by setting the `writer` attribute/initializer parameter to `None`.  The progress bar and columns are set with the `progress_bar` and `progress_bar_col` attributes.

In [ ]:
# set the number of epochs and learning rate, which are both model parameters
facade.epochs = 20
facade.learning_rate = .01
# train and test the model
facade.train()
facade.test()
# display the results in this cell
facade.write_result()

## Network parameters

While we're at it, let's also adjust the drop out, which is a network settings, to see if we can get better results.  Also note that the model converged pretty late indicating we aren't over training, so add more epochs.

In [ ]:
# turn off output so we can get just the final results later with `write_result`
facade.writer = None
# set network parameter `dropout` and model settings to achieve better performance
facade.dropout = 0.1
# set lower learning rate and compensate with epochs in case learning is slower
facade.epochs = 1000
facade.learning_rate = .005
# train and test again
facade.train()
facade.test()
# display the results in this cell
facade.write_result()
# plot results
facade.plot_result()
# now since we like our results, save them to disk
facade.persist_result()

## Predictions

The executor contains the results from the last run, in additional to saving it.  In our case, we have the same instance of the model we just tested, which contains not only the performance metrics, but the predictions themselves.  Use `get_predictions` to get a Pandas `pd.DataFrame` for the results.

In [ ]:
from iris.model import IrisDataPoint

# optionally, we can transform the data point instance used, otherwise it defaults to `str`
def map_data_point(dp: IrisDataPoint):
    """Map the data point's Pandas row information (pd.Series) to key/value string.
    
    :param dp: the data point created by the ``iris_dataset_stash`` as defined in the configuration
    
    """
    s = ', '.join(map(lambda x: f'{x[0]}={x[1]}', dp.row.items()))
    return dp.row['sepal_length'], dp.row['sepal_width'], dp.row['petal_length'], dp.row['petal_width']
facade.get_predictions(column_names=['sepal length', 'sepal width', 'petal length', 'petal width'], transform=map_data_point)

In [ ]:
from zensols.persist import dealloc
from zensols.deeplearn.model import ModelFacade
from pathlib import Path

# deallocate the previous facade
facade.deallocate()
path = Path('../target/iris/results/iris-1.model')
# create a new facade wrapped in a deallocation block
# (it will be deallocated even in an error generated in the block)
with dealloc(ModelFacade.load_from_path(path)) as facade:
    facade.write_result()
# note that no test results are given since this model was saved during
# training after achieving the lowest validation loss

In [ ]:
# we can also revive the model as a facade to test, then we'll get the test results
with dealloc(ModelFacade.load_from_path(path)) as facade:
    facade.writer = None
    facade.test()
    facade.write_result(include_converged=True)